# Conformation finder using MD
Since we can quickly generate force fields we could use those to run dynamics and identify new low energy conformations of flexible molecules

In [ ]:
import veloxchem as vlx

In [ ]:
#Lynparza
molecule = vlx.Molecule.read_smiles('C1CC1C(=O)N2CCN(CC2)C(=O)C3=C(C=CC(=C3)CC4=NNC(=O)C5=CC=CC=C54)F')
#Paxlovid
#molecule = vlx.Molecule.read_smiles('CC1([C@@H]2[C@H]1[C@H](N(C2)C(=O)[C@H](C(C)(C)C)NC(=O)C(F)(F)F)C(=O)N[C@@H](C[C@@H]3CCNC3=O)C#N)C')

molecule.show()
basis = vlx.MolecularBasis.read(molecule, 'def2-SVP')
basis.get_dimension_of_basis()
print(molecule.get_xyz_string())

This time we generate a forcefield using custom SCF settings.

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "blyp"
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.conv_thresh = 1e-3
scf_drv.dispersion = True
scf_drv.timing = True
results = scf_drv.compute(molecule, basis)
resp_drv = vlx.RespChargesDriver()
charges = resp_drv.compute(molecule, basis, results, 'resp')

In [ ]:
conf_generator = vlx.ConformerGenerator()
conf_generator.resp_charges = False
conf_generator.partial_charges = charges
conformers = conf_generator.generate(molecule)

In [ ]:
conf_generator.show_global_minimum()

The force field is then used in a high temp MD-simulation where a number of snapshots (snapshots=50) are minimized.

In [ ]:
molecule = conformers['molecules'][0]
basis = vlx.MolecularBasis.read(molecule, 'def2-SVP')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "blyp"
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.conv_thresh = 1e-3
scf_drv.dispersion = True
scf_drv.timing = True
results2 = scf_drv.compute(molecule, basis)

In [ ]:
print((results2['scf_energy']-results['scf_energy'])*2625.5)
